In [ ]:
ALTER SESSION SET TIMEZONE = 'Europe/London';

In [ ]:
import os
import requests
from azure.identity import ClientSecretCredential
import pandas as pd
from json import loads, dumps
try:
    from snowflake.snowpark.context import get_active_session

    session = get_active_session()

    results_tenant_id = session.sql("SELECT prod.raw.ADO_DEFENDER_TENANT_ID()").collect()
    tenant_id = results_tenant_id[0][0]

    results_client_id = session.sql("SELECT prod.raw.ADO_DEFENDER_CLIENT_ID()").collect()
    client_id = results_client_id[0][0]

    results_client_secret = session.sql("SELECT prod.raw.ADO_DEFENDER_CLIENT_SECRET()").collect()
    client_secret = results_client_secret[0][0]

    results_subscription_id = session.sql("SELECT prod.raw.ADO_DEFENDER_SUBSCRIPTION_ID()").collect()
    subscription_id = results_subscription_id[0][0]
except:
    from dotenv import load_dotenv
    load_dotenv()
    tenant_id = os.getenv('tenant_id')
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
    subscription_id = os.getenv('subscription_id')


https://learn.microsoft.com/en-us/rest/api/defenderforcloud-composite/alerts/list?view=rest-defenderforcloud-composite-stable&tabs=HTTP

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/alerts?api-version=2022-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}



all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


alerts = pd.DataFrame(all_allerts)

print(f"Alerts: {len(alerts)}")
alerts

# # Make the API call
# response = requests.get(url, headers=headers)
# data = response.json()
# alerts = pd.DataFrame(data['value'])
# alerts

In [ ]:

temp_alerts = 'TEMP_ALERTS'

session.write_pandas(alerts,
                         table_name=temp_alerts,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
CREATE OR REPLACE TABLE prod.raw.azure_defender_alerts AS 
SELECT
  "id",
  "name",
  "type", 
  PARSE_JSON("properties"):"alertDisplayName"::string      AS alert_display_name,
  PARSE_JSON("properties"):"alertType"::string              AS alert_type,
  PARSE_JSON("properties"):"alertUri"::string               AS alert_uri,
  PARSE_JSON("properties"):"compromisedEntity"   ::string   AS compromised_entity,
  PARSE_JSON("properties"):"correlationKey"   ::string      AS correlation_key,
  PARSE_JSON("properties"):"description"    ::string        AS description,
  PARSE_JSON("properties"):"endTimeUtc"  ::string           AS end_time_utc,
  PARSE_JSON("properties"):"intent"    ::string             AS intent,
  PARSE_JSON("properties"):"isIncident"  ::string           AS is_incident,
  PARSE_JSON("properties"):"productName"    ::string        AS product_name,
  PARSE_JSON("properties"):"severity" ::string              AS severity,
  PARSE_JSON("properties"):"startTimeUtc"   ::string        AS start_time_utc,
  PARSE_JSON("properties"):"status" ::string                AS status,
  PARSE_JSON("properties"):"systemAlertId" ::string         AS system_alert_id,
  PARSE_JSON("properties"):"timeGeneratedUtc"::timestamp_tz       AS time_generated,
  PARSE_JSON("properties"):"entities" ::variant              AS entities,
  PARSE_JSON("properties"):"extendedProperties"  ::variant   AS extended_properties,
  PARSE_JSON("properties"):"remediationSteps"::variant       AS remediation_steps,
PARSE_JSON("properties"):"resourceIdentifiers" ::variant   AS resource_identifiers,

FROM TEMP_ALERTS;

https://learn.microsoft.com/en-us/rest/api/advisor/recommendations/list?view=rest-advisor-2025-01-01&tabs=HTTP

In [ ]:
# # Authenticate
# credential = ClientSecretCredential(tenant_id, client_id, client_secret)
# token = credential.get_token("https://management.azure.com/.default")

# # API endpoint
# url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Advisor/recommendations?api-version=2025-01-01"

# # Headers with access token
# headers = {
#     "Authorization": f"Bearer {token.token}",
#     "Content-Type": "application/json"
# }


# all_recommendations= []

# response = requests.get(url, headers=headers)
# data = response.json()
# all_recommendations.extend(data.get('value', []))

# while "nextLink" in data:
#     next_url = data["nextLink"]
#     response = requests.get(next_url, headers=headers)
#     data = response.json()
#     all_recommendations.extend(data.get('value', []))


# recommendations = pd.DataFrame(all_recommendations)

# print(f"Recommendations: {len(recommendations)}")
# recommendations

# # # Make the API call
# # response = requests.get(url, headers=headers)
# # data = response.json()
# # recommendations = pd.DataFrame(data['value'])
# # recommendations

In [ ]:

# temp_recommendations = 'TEMP_RECOMM'

# session.write_pandas(recommendations,
#                          table_name=temp_recommendations,
#                          auto_create_table=True,
#                          overwrite=True,
#                          table_type="temporary")

In [ ]:
-- -- CREATE OR REPLACE TABLE prod.raw.azure_defender_recommendations AS 

-- select
-- "name" as id,
-- parse_json("properties"):"category"::text as category,
-- parse_json("properties"):"impact"::text as impact,
-- parse_json("properties"):"impactedField"::text as impactedField,
-- parse_json("properties"):"impactedValue"::text as impactedValue,
-- parse_json("properties"):"lastUpdated"::text as lastUpdated,
-- parse_json("properties"):"recommendationTypeId"::text as recommendationTypeId,
-- parse_json("properties"):"resourceMetadata"."resourceId"::text as resourceId,
-- parse_json("properties"):"shortDescription"."problem"::text as shortDescription_problem,
-- parse_json("properties"):"shortDescription"."solution"::text as shortDescription_solution,
-- parse_json("properties"):"extendedProperties"."assessmentKey"::text as extendedProperties,
-- parse_json("properties"):"extendedProperties"::text as extendedProperties,
-- parse_json("properties"):"extendedProperties"::text as extendedProperties,
-- parse_json("properties"):"extendedProperties"::text as extendedProperties,
-- parse_json("properties")::variant as properties,
-- parse_json("properties"):"resourceMetadata"::text as resourceMetadata,
-- from TEMP_RECOMM

https://learn.microsoft.com/en-us/rest/api/defenderforeasm/dataplanepreview/tasks/list-task?view=rest-defenderforeasm-dataplanepreview-2024-10-01-preview&tabs=HTTP

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/tasks?api-version=2015-06-01-preview"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}



all_tasks= []

response = requests.get(url, headers=headers)
data = response.json()
all_tasks.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_tasks.extend(data.get('value', []))


tasks = pd.DataFrame(all_tasks)

print(f"Tasks: {len(tasks)}")
tasks

# # Make the API call
# response = requests.get(url, headers=headers)
# data = response.json()
# tasks = pd.DataFrame(data['value'])
# tasks

In [ ]:

temp_tasks= 'TEMP_TASKS'

session.write_pandas(tasks,
                         table_name=temp_tasks,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
CREATE OR REPLACE TABLE prod.raw.azure_defender_tasks AS 
select 
parse_json("properties") as prosp,
parse_json("properties"):"creationTimeUtc"::text as creationTimeUtc,
parse_json("properties"):"lastStateChangeTimeUtc"::text as lastStateChangeTimeUtc,
parse_json("properties"):"securityTaskParameters"."name"::text as securityTaskParameters,
parse_json("properties"):"securityTaskParameters"."resourceId"::text as resourceId,
parse_json("properties"):"securityTaskParameters"."uniqueKey"::text as uniqueKey,
parse_json("properties"):"state"::text as state,
parse_json("properties"):"subState"::text as subState
from TEMP_TASKS

https://learn.microsoft.com/en-us/rest/api/defenderforcloud-composite/assessments/list?view=rest-defenderforcloud-composite-stable&tabs=HTTP

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/assessments?api-version=2025-05-04"

# Nagłówki z tokenem dostępu
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

# Pobieranie wszystkich alertów z obsługą paginacji
all_assessments= []

response = requests.get(url, headers=headers)
data = response.json()
all_assessments.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_assessments.extend(data.get('value', []))

# Tworzenie DataFrame
assessments = pd.DataFrame(all_assessments)

# Wyświetlenie liczby pobranych alertów i przykładowych danych
print(f"Assessments: {len(assessments)}")


assessments['subAssessmentsLink'] = assessments['properties'].apply(
    lambda x: x.get('additionalData', {}).get('subAssessmentsLink') if isinstance(x, dict) else None
)
assessments['displayName'] = assessments['properties'].apply(lambda x: x.get('displayName') if isinstance(x, dict) else None)

assessments['assessmentType'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('assessmentType') if isinstance(x, dict) else None
)
assessments['categories'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('categories') if isinstance(x, dict) else None
)
assessments['cloudProviders'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('cloudProviders') if isinstance(x, dict) else None
)

assessments['description'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('description') if isinstance(x, dict) else None
)

assessments['managementProvider'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('managementProvider') if isinstance(x, dict) else None
)

assessments['policyDefinitionId'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('policyDefinitionId') if isinstance(x, dict) else None
)

# assessments['publishDates'] = assessments['properties'].apply(
#     lambda x: x.get('metadata', {}).get('publishDates') if isinstance(x, dict) else None
# )

assessments['recommendationCategory'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('recommendationCategory') if isinstance(x, dict) else None
)

assessments['remediationDescription'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('remediationDescription') if isinstance(x, dict) else None
)

assessments['securityCategories'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('securityCategories') if isinstance(x, dict) else None
)

assessments['securityIssue'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('securityIssue') if isinstance(x, dict) else None
)

assessments['techniques'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('techniques') if isinstance(x, dict) else None
)

assessments['threats'] = assessments['properties'].apply(
    lambda x: x.get('metadata', {}).get('threats') if isinstance(x, dict) else None
)

assessments['resourceDetails_Id'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('Id') if isinstance(x, dict) else None
)
assessments['NativeResourceId'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('NativeResourceId') if isinstance(x, dict) else None
)


assessments['ResourceId'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('ResourceId') if isinstance(x, dict) else None
)


assessments['ResourceName'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('ResourceName') if isinstance(x, dict) else None
)


assessments['ResourceType'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('ResourceType') if isinstance(x, dict) else None
)


assessments['Source'] = assessments['properties'].apply(
    lambda x: x.get('resourceDetails', {}).get('Source') if isinstance(x, dict) else None
)


assessments['status_code'] = assessments['properties'].apply(
    lambda x: x.get('status', {}).get('code') if isinstance(x, dict) else None
)

assessments['first_Evaluation_Date'] = assessments['properties'].apply(
    lambda x: x.get('status', {}).get('firstEvaluationDate') if isinstance(x, dict) else None
)

assessments['status_Change_Date'] = assessments['properties'].apply(
    lambda x: x.get('status', {}).get('statusChangeDate') if isinstance(x, dict) else None
)

# assessments['assessmentStatus'] = assessments['properties'].apply(
#     lambda x: x.get('statusPerInitiative', {}).get('assessmentStatus') if isinstance(x, dict) else None
# )

# # detailed assessments didnt give any additional data
# detailed_assessments = []

# for index, row in assessments.iterrows():
#         # Wyciągnij resourceId i assessmentName
#         resource_id = row['properties']['resourceDetails']['Id']
#         assessment_name = row['name']
        
#         # Zbuduj URL
#         detail_url = f"https://management.azure.com{resource_id}/providers/Microsoft.Security/assessments/{assessment_name}?api-version=2021-06-01"
        
#         # Wykonaj zapytanie
#         detail_response = requests.get(detail_url, headers=headers)
        
#         if detail_response.status_code == 200:
#             detailed_assessments.append(detail_response.json())
#         else:
#             print(f"Błąd dla assessment {assessment_name}: {detail_response.status_code}")
            

# # Tworzenie DataFrame z szczegółami
# detailed_df = pd.DataFrame(detailed_assessments)
# detailed_df

assessments = assessments.drop(columns=["tags"
                                        , "properties"
                                       ])

assessments

In [ ]:

temp_assessments = 'TEMP_ASSESSMENTS'

session.write_pandas(assessments,
                         table_name=temp_assessments,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.azure_defender_assessments as 

select  
-- parse_json("properties"):"additionalData"::string additionalData,
-- parse_json("properties")::variant as properties,
-- parse_json("properties"):"status"."code"::string status_code,
-- parse_json("properties"):"status"."firstEvaluationDate"::string first_Evaluation_Date,
-- parse_json("properties"):"status"."statusChangeDate"::string status_Change_Date,
-- parse_json("properties"):"displayName"::string as display_name,
-- parse_json("properties"):"resourceDetails"."ResourceName"::string as Resource_name,
-- parse_json("properties"):"resourceDetails"."ResourceProvider"::string as Resource_Provider,
-- parse_json("properties"):"resourceDetails"."ResourceType"::string as ResourceType,
-- parse_json("properties"):"resourceDetails"."Source"::string as Source,
-- parse_json("properties"):"additionalData"."maxCvss30Score"::string as maxCvss30Score,

-- "type", "id" , "name",
*
from TEMP_ASSESSMENTS

https://learn.microsoft.com/en-us/rest/api/defenderforcloud-composite/assessments-metadata/list-by-subscription?view=rest-defenderforcloud-composite-stable&tabs=HTTP

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/assessmentMetadata?api-version=2021-06-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_assessments_md= []

response = requests.get(url, headers=headers)
data = response.json()
all_assessments_md.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_assessments_md.extend(data.get('value', []))


assessments_md = pd.DataFrame(all_assessments_md)
assessments_md = assessments_md.drop(columns='tags')
print(f"Assessments metadata: {len(assessments_md)}")
assessments_md


In [ ]:
session = get_active_session()
temp_assessments_md = 'TEMP_ASSESSMENTS_MD'

session.write_pandas(assessments_md,
                         table_name=temp_assessments_md,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
CREATE OR REPLACE TABLE prod.raw.azure_defender_assessments_metadata AS 

select 
parse_json("properties"):"displayName"::text as displayname,
parse_json("properties"):"assessmentType"::text as assessmentType,
parse_json("properties"):"categories"::variant as categories,
parse_json("properties"):"cloudProviders"::variant as cloudProviders,
parse_json("properties"):"description"::text as description,
parse_json("properties"):"implementationEffort"::text as implementationEffort,
parse_json("properties"):"managementProvider"::text as managementProvider,
parse_json("properties"):"policyDefinitionId"::text as policyDefinitionId,
parse_json("properties"):"publishDates"."ga"::text as publishDates_ga,
parse_json("properties"):"publishDates"."public"::text as publishDates_public,
parse_json("properties"):"recommendationCategory"::text as recommendationCategory,
parse_json("properties"):"remediationDescription"::text as remediationDescription,
parse_json("properties"):"securityCategories"::text as securityCategories,
parse_json("properties"):"securityIssue"::text as securityIssue,
parse_json("properties"):"severity"::text as severity,
parse_json("properties"):"tactics"::variant as tactics,
parse_json("properties"):"techniques"::variant as techniques,
parse_json("properties"):"threats"::text as variant,
parse_json("properties"):"userImpact"::text as userImpact,
parse_json("properties") as properties

from TEMP_ASSESSMENTS_MD

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/allowedConnections?api-version=2020-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)
data = response.json()
print(f"{response}")
data


In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/providers/Microsoft.Security/secureScoreControlDefinitions?api-version=2020-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}


all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


secureScoreControlDefinitions = pd.DataFrame(all_allerts)

print(f"secureScoreControlDefinitions: {len(secureScoreControlDefinitions)}")
secureScoreControlDefinitions

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/secureScoreControls?api-version=2020-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


secureScoreControls = pd.DataFrame(all_allerts)

print(f"secureScoreControls: {len(secureScoreControls)}")
secureScoreControls

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/secureScores?api-version=2020-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


secureScores = pd.DataFrame(all_allerts)

print(f"secureScores: {len(secureScores)}")
secureScores


In [ ]:

temp_secureScores = 'TEMP_SECURE_SCORES'

session.write_pandas(secureScores,
                         table_name=temp_secureScores,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/securitySolutions?api-version=2020-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


securitySolutions = pd.DataFrame(all_allerts)

print(f"securitySolutions: {len(securitySolutions)}")
securitySolutions


In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/securityStandards?api-version=2024-08-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_allerts= []

response = requests.get(url, headers=headers)
data = response.json()
all_allerts.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allerts.extend(data.get('value', []))


securityStandards = pd.DataFrame(all_allerts)

print(f"securityStandards: {len(securityStandards)}")
securityStandards

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/pricings?api-version=2024-01-01"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_pricings= []

response = requests.get(url, headers=headers)
data = response.json()
all_pricings.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_pricings.extend(data.get('value', []))


pricings = pd.DataFrame(all_pricings)
pricings


In [ ]:

temp_pricings = 'TEMP_PRICINGS'

session.write_pandas(pricings,
                         table_name=temp_pricings,
                         auto_create_table=True,
                         overwrite=True,
                         table_type="temporary")

In [ ]:
create or replace table prod.raw.azure_defender_pricings as 
select 
"id",
"name",
"type",
parse_json("properties"):"enablementTime"::timestamp_tz as enablementTime,
parse_json("properties"):"extensions"::text as extensions,
parse_json("properties"):"freeTrialRemainingTime"::text as freeTrialRemainingTime,
parse_json("properties"):"pricingTier"::text as pricingTier,
parse_json("properties"):"resourcesCoverageStatus"::text as resourcesCoverageStatus,
parse_json("properties"):"deprecated"::boolean as deprecated,
parse_json("properties"):"subPlan"::text as subPlan,
parse_json("properties"):"replacedBy"::text as replacedBy,
parse_json("properties")::text as properties
from TEMP_PRICINGS

In [ ]:
# Authenticate
credential = ClientSecretCredential(tenant_id, client_id, client_secret)
token = credential.get_token("https://management.azure.com/.default")

# API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.Security/healthReports?api-version=2023-05-01-preview"

# Headers with access token
headers = {
    "Authorization": f"Bearer {token.token}",
    "Content-Type": "application/json"
}

all_allowedConnections= []

response = requests.get(url, headers=headers)
data = response.json()
all_allowedConnections.extend(data.get('value', []))

while "nextLink" in data:
    next_url = data["nextLink"]
    response = requests.get(next_url, headers=headers)
    data = response.json()
    all_allowedConnections.extend(data.get('value', []))


customRecommendations = pd.DataFrame(all_allowedConnections)
# allowedConnections = allowedConnections.drop(columns='tags')
print(f"Assessments metadata: {len(customRecommendations)}")
customRecommendations
